## Создание индекса FAISS для бота

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import sys

# Получаем абсолютный путь к корневой директории проекта (директория выше текущей)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Добавляем корневую директорию в sys.path
if root_path not in sys.path:
    sys.path.append(root_path)

In [9]:
import torch
from models.clap_encoder import CLAP_Encoder
import faiss
from faiss import write_index, read_index
from constants.index_classes import class_checkpoint_combinations

In [10]:
CLAP_CKPT_PATH = '../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt'

In [11]:
device = torch.device('cuda')
query_encoder = CLAP_Encoder(pretrained_path = CLAP_CKPT_PATH).eval().to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
combined_classes = class_checkpoint_combinations
# Генерируем эмбеддинги для всех классов
all_classes = [cls for cls, _ in combined_classes]
all_embeddings = query_encoder.get_query_embed(modality='text', text=all_classes).cpu()
# Создаем индекс FAISS
index = faiss.IndexHNSWFlat(512, 32)
index.add(all_embeddings)

Проверяем что бд находит правильный вектор (ожидаем dog)

In [13]:
from separator import find_nearest_embedding

query_embedding = query_encoder.get_query_embed(modality='text', text=['bark']).cpu()
predicted_class, ckpt_path, distance = find_nearest_embedding(combined_classes, query_embedding, index, threshold=1.0)
print(predicted_class, ckpt_path, distance)

meepo checkpoints/final/dota2/lora_embeddings/final.ckpt 0.214363


Сохраняем индекс

In [14]:
index_path = "combined.idx"
write_index(index, index_path)

Проверяем что он работает

In [15]:
index_from_disk = read_index(index_path)
query_embedding = query_encoder.get_query_embed(modality='text', text=['bark']).cpu()
predicted_class, ckpt_path, distance = find_nearest_embedding(combined_classes, query_embedding, index_from_disk, threshold=1.0)
print(predicted_class, ckpt_path, distance)

meepo checkpoints/final/dota2/lora_embeddings/final.ckpt 0.214363
